DB_MONITOR

#Procura arquivos com base na lista gerada pelas certidões cadastradas por clientes df_cert_config['CERTIDOES_CADASTRADAS']
e salva/atualiza dB_monitor

In [ ]:
import pandas as pd
from openpyxl import load_workbook
import os
from datetime import datetime, timedelta
import pdfplumber
import re

In [259]:
gestao_monitoramento = 'gestao_monitoramento.xlsx'
# salva formatação
workbook = load_workbook(gestao_monitoramento)

def certidoes_clientes_ativo(df_clientes, df_cert_config):
    try:
        df_clientes_expandido = df_clientes.assign(
            CERTIDAO_ID=df_clientes['CERTIDOES_CADASTRADAS'].str.split(', ')
        ).explode('CERTIDAO_ID')

        df_clientes_certidoes = pd.merge(df_clientes_expandido,
                                    df_cert_config,
                                    on='CERTIDAO_ID', 
                                    how='left')[['CERTIDAO_ID', 'CLIENTE_ID', 'ATIVO', 'PASTA_DESTINO', 'NOME_ARQUIVO']]

        # filtrando apenas onde o 'ATIVO' é True
        df_clientes_certidoes = df_clientes_certidoes.loc[df_clientes_certidoes['ATIVO'] == True]
        
        print('df_clientes_certidoes criado com sucesso')
        return df_clientes_certidoes
    except Exception as e:
        print(f'erro ao gerar df_clientes_certidoes {e}')
        return None

def verificar_existencia_arquivos(df_clientes_certidoes):
    try:
        resultados = []
        pastas_grupo = df_clientes_certidoes.groupby('PASTA_DESTINO')

        for pasta_destino, grupo in pastas_grupo:
            # verificar se o diretório existe
            if not os.path.isdir(pasta_destino):
                for _, row in grupo.iterrows():
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'ARQUIVO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
                continue

            # lista todos os arquivos da pasta_destino
            arquivos_na_pasta = os.listdir(pasta_destino)

            for _, row in grupo.iterrows():
                nome_arquivo = row['NOME_ARQUIVO']
                arquivos_encontrados = []

                for arquivo in arquivos_na_pasta:
                    if nome_arquivo in arquivo and arquivo.endswith('.pdf'):
                        arquivo_pdf = os.path.join(arquivo)
                        arquivos_encontrados.append(arquivo_pdf)

                # Adicionar uma entrada para cada arquivo encontrado
                if arquivos_encontrados:
                    for arquivo_pdf in arquivos_encontrados:
                        resultados.append({
                            'CERTIDAO_ID': row['CERTIDAO_ID'],
                            'CLIENTE_ID': row['CLIENTE_ID'],
                            'EXISTE': True,
                            'ARQUIVO': arquivo_pdf,
                            'PASTA_DESTINO': pasta_destino
                        })
                else:
                    # Se nenhum arquivo foi encontrado, adicionar como não encontrado
                    resultados.append({
                        'CERTIDAO_ID': row['CERTIDAO_ID'],
                        'CLIENTE_ID': row['CLIENTE_ID'],
                        'EXISTE': False,
                        'ARQUIVO': None,
                        'PASTA_DESTINO': pasta_destino
                    })
        #cria df com os resultados encontrados
        df_clientes_certidoes_resultado = pd.DataFrame(resultados)
        return df_clientes_certidoes_resultado
    except Exception as e:
        print(f'Erro ao verificar a existência dos arquivos {e}')

# Carrega dataframes
df_clientes = pd.read_excel(gestao_monitoramento, sheet_name='config_clientes', index_col=None)
df_cert_config = pd.read_excel(gestao_monitoramento, sheet_name='config_certidoes', index_col=None)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')


df_clientes_certidoes = certidoes_clientes_ativo(df_clientes, df_cert_config)

df_clientes_certidoes_resultado = verificar_existencia_arquivos(df_clientes_certidoes)



df_clientes_certidoes criado com sucesso


In [263]:

def arquivos_novos_dbmonitor(df_monitor, df_clientes_certidoes_resultado):
    horario_atual = datetime.now().replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')

    certidao_nao_encontrada = []
    certidao_adicionada = []

    # Itera sobre cada linha do DataFrame df_resultados
    for _, row in df_clientes_certidoes_resultado.iterrows():
        certidao_id = row['CERTIDAO_ID']
        cliente_id = row['CLIENTE_ID']
        
        if row['EXISTE']:
            arquivo = row['ARQUIVO']
            pasta_destino = row['PASTA_DESTINO']
            
            # Verifica existencia de linha com CERTIDAO_ID, CLIENTE_ID e ARQUIVO iguais
            condicao_existente = (
                (df_monitor['CERTIDAO_ID'] == certidao_id) &
                (df_monitor['CLIENTE_ID'] == cliente_id) &
                (df_monitor['ARQUIVO'] == arquivo)
            )
            
            if df_monitor[condicao_existente].empty:
                novo_dado = {
                    'CERTIDAO_ID': certidao_id,
                    'CLIENTE_ID': cliente_id,
                    'PASTA_DESTINO': pasta_destino,
                    'ARQUIVO': arquivo,
                    'STATUS': 'adicionado',
                    'DATA_INICIO': None,
                    'DATA_VENCIMENTO': None,
                    'NUMERO_CERTIDAO': None,
                    'NADA_CONSTA': None,
                    'CREATED_AT': None,
                    'UPDATED_AT': horario_atual
                }
                certidao_adicionada.append(novo_dado)
                df_monitor = pd.concat([df_monitor, pd.DataFrame([novo_dado])], ignore_index=True)

        # Condição para quando 'EXISTE' é False
        else:
            certidao_nao_encontrada.append({
                'CERTIDAO_ID': certidao_id,
                'CLIENTE_ID': cliente_id,
                'PASTA_DESTINO': row['PASTA_DESTINO'] if pd.notnull(row['PASTA_DESTINO']) else None,
                'TIME_STAMP': horario_atual
            })

    # Exibe o conteúdo atualizado de df_monitor e a lista de certidões não encontradas
    print(f'Certidões não encontradas:{len(certidao_nao_encontrada)}')
    print(certidao_nao_encontrada)
    print('df_monitor atualizado')
    print(f'Certidões adicionadas:{len(certidao_adicionada)}')

    return df_monitor

In [266]:
df_monitor = arquivos_novos_dbmonitor(
    df_monitor, df_clientes_certidoes_resultado)

Certidões não encontradas:6
[{'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'AUTOLTDA', 'PASTA_DESTINO': 'clientes/autodata/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}, {'CERTIDAO_ID': 'GDF-CND', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BIGBOX', 'PASTA_DESTINO': 'clientes/bigbox/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'BOTMEI', 'PASTA_DESTINO': 'clientes/botmei/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'DROSARIO', 'PASTA_DESTINO': 'clientes/drosario/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}, {'CERTIDAO_ID': 'TESTE-TESTE', 'CLIENTE_ID': 'TLTDA', 'PASTA_DESTINO': 'clientes/testeltda/', 'TIME_STAMP': datetime.datetime(2024, 11, 14, 0, 53, 9)}]
df_monitor atualizado
Certidões adicionadas:0


In [265]:
import openpyxl

def atualizar_planilha(gestao_monitoramento, workbook, df_monitor):
    # Carrega o arquivo Excel
    
    # Seleciona a aba 'db_monitor'
    sheet = workbook['db_monitor']

    # Limpa todas as linhas da planilha (exceto o cabeçalho, linha 1)
    for row in sheet.iter_rows(min_row=2, max_col=sheet.max_column, max_row=sheet.max_row):
        for cell in row:
            cell.value = None

    # Preenche os dados do DataFrame a partir da linha 2
    for index, row in df_monitor.iterrows():
        for col_num, value in enumerate(row, start=1):  # start=1 para a primeira coluna (A)
            sheet.cell(row=index + 2, column=col_num, value=value)  # Inicia na linha 2, logo abaixo do cabeçalho

    # Salva as alterações no arquivo Excel
    workbook.save(gestao_monitoramento)
    print("Arquivo Excel atualizado com os dados do DataFrame!")

atualizar_planilha(gestao_monitoramento, workbook, df_monitor)


Arquivo Excel atualizado com os dados do DataFrame!


### ACESSANDO ARQUIVOS PARA EXTRAIR DADOS E PREENCHER NO DB_MONITOR


In [ ]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
df_monitor.info()
df_monitor.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      55 non-null     object        
 1   CLIENTE_ID       55 non-null     object        
 2   PASTA_DESTINO    55 non-null     object        
 3   ARQUIVO          55 non-null     object        
 4   STATUS           55 non-null     object        
 5   DATA_INICIO      55 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  55 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  55 non-null     object        
 8   NADA_CONSTA      55 non-null     bool          
 9   CREATED_AT       55 non-null     datetime64[ns]
 10  UPDATED_AT       55 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 4.5+ KB


,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-19_CR FGTS.pdf,monitorando,2024-10-21,2024-11-19,2024102105285887458243,True,2024-11-13 20:08:57,2024-11-13 22:07:17.242
1,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-08_CR FGTS.pdf,VENCIDA,2024-10-03,2024-11-08,J0ER9QQR4U,True,2024-11-13 20:08:58,2024-11-14 01:34:47.709
2,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-12-01_CN Falencia e Recuperacao.pdf,monitorando,2024-11-01,2024-12-01,2024.CTD.ZZQ3.G1NW.0B85.GEDK.UR8P,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601
3,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-11-04_CN Falencia e Recuperacao.pdf.pdf,VENCIDA,2024-10-05,2024-11-04,091O5ZAMKM,True,2024-11-13 20:08:58,2024-11-14 01:34:47.709
4,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-12-04_CN Correcional (ePAD, CGU-PJ, CEIS,...",monitorando,2024-11-04,2024-12-04,YLkevKTcTH7aFG1S8seQ,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601


In [334]:
def data_criacao_pdf(pdf_path):
    try:
        timestamp =  os.path.getctime(pdf_path)
        created_at = datetime.fromtimestamp(timestamp).replace(microsecond=0)#.strftime('%Y-%m-%d %H:%M:%S')
        return created_at
    except Exception as e:
        print(f'Erro ao obter a data de criação do arquivo: {e}')
        return None


def extrair_data_inicio(texto_pdf):
    try:
        padrao_regex = [
            r'Validade:(\d{2}/\d{2}/\d{4})',
            r'Emitida gratuitamente pela internet em:\s*(\d{2}/\d{2}/\d{4})',
            r'Gerado em:\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Emitida às\s*\d{2}:\d{2}:\d{2}\s*do dia\s*(\d{2}/\d{2}/\d{4})',
            r'Expedição:\s*(\d{2}/\d{2}/\d{4})\s*, às\s*\d{2}:\d{2}:\d{2}',
            r'Este documento foi emitido no dia\s*(\d{2}/\d{2}/\d{4})',
            r'Certidão emitida via internet em\s*(\d{2}/\d{2}/\d{4})\s*às\s*\d{2}:\d{2}:\d{2}'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                data_encontrada = par.group(1)
                data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                # data_formatada = data_objeto.strftime('%Y-%m-%d')
                return data_objeto

        print('Data de início não encontrada.')
        return None

    except Exception as e:
        print(f'Erro ao extrair data de início do arquivo: {e}')
        return None

def extrair_numero_certidao(texto_pdf, row):
    try:
        padrao_regex = [
            r'Certificação Número:\s*([\w./]+)',  # Ex: "Certificação Número: 024102105285887458243"
            r'Selo digital de segurança:\s*([\w./]+)',  # Ex: "Selo digital de segurança: 2024.CTD.ZZQ3.G1NW.0B85.GEDK.UR8P"
            r'número de controle\s*\n*([\w./]+)',  # Ex: "número de controle\n672D.DF85.D1C3.5589"
            r'Código de controle da certidão:\s*([\w./]+)',  # Ex: "Código de controle da certidão: YLkevKTcTH7aFG1S8seQ"
            r'Certidão nº:\s*([\w./]+)',  # Ex: "Certidão nº: 76418628/2024"
            r'CERTIDÃO Nº:\s*([\w./]+)'
        ]
        if row['CERTIDAO_ID'] == 'GDF-DIF':
            return '-não existe-'
        else:
            for padrao in padrao_regex:
                par = re.search(padrao, texto_pdf)
                if par:
                    numero_certidao = par.group(1)
                    return numero_certidao


    except Exception as e:
        print(f'Erro ao extrair número da certidão: {e}')
        return None


def extrair_nada_consta(texto_pdf):
    try:
        padrao_regex = [
            r'NÃO\s*CONSTA',                    # "NÃO CONSTA"
            r'NADA\s*CONSTA',                   # "NADA CONSTA"
            r'encontra-se em situação regular', # "encontra-se em situação regular"
            r'não\s*constam',                   # "não constam"
            r'NÃO\s*CONSTAM',                   # "NÃO CONSTAM"
            r'ATIVO\s*\d{2}/\d{2}/\d{4}',       # Ativo - verifica cadastro ativo
            r'tem\s+os\s+mesmos\s+efeitos\s+da\s+certidão\s+negativa\.', # positiva com efeito de negativa
            r'certidão\s+positiva\s+de\s+débitos\s+com\s+efeito\s+de\s+negativa',
            r'certidão\s+positiva\s+com\s+efeito\s+de\s+negativa'
        ]

        for padrao in padrao_regex:
            par = re.search(padrao, texto_pdf)
            if par:
                return True
        
        return False
    
    except Exception as e:
        print(f'Erro ao extrair Nada Consta da certidão: {e}')
        return False


def extrair_data_vencimento(texto_pdf, data_inicio, row):
    try:
        padrao_regex = [
            r"Validade:\s*\d{2}/\d{2}/\d{4}\s*a\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Validade:21/10/2024 a 19/11/2024" (pegar a segunda data)
            r"Validade:\s*(\d{2}/\d{2}/\d{4})\s*",  # Ex: "Validade: 03/05/2025 -"
            r"Válida até\s*(\d{2}/\d{2}/\d{4})",  # Ex: "Válida até 14/04/2025"
            r"Válida até\s*(\d{2}) de (\w+) de (\d{4})",  # Ex: "Válida até 02 de fevereiro de 2025"
            r", com validade até o dia\s*(\d{2}/\d{2}/\d{4})"  # Ex: ", com validade até o dia 04/12/2024"
        ]
        
        meses = {
            'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
            'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
            'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
        }

        for padrao in padrao_regex:
            if row['CERTIDAO_ID'] in ('GDF-DIF','TJDFT-CNFR','CNJ-CNIADM'):
                print('TENTANDDO NOVA DATA AQUI')
                # data_objeto = datetime.strptime(data_inicio, '%Y-%m-%d')
                data_vencimento = data_inicio + timedelta(days=30)
                print(f'data_vencimento: {data_vencimento}')
                return data_vencimento
            
            par = re.search(padrao, texto_pdf)
            if par:
                # data por extenso
                if len(par.groups()) == 3:
                    dia, mes_extenso, ano = par.groups()
                    mes = meses.get(mes_extenso.lower()) # pega no dicionario
                    data_str = (f'{ano}-{mes}-{dia}')
                    data = datetime.strptime(data_str, '%Y-%m-%d')
                    return data
                else:
                    data_encontrada = par.group(1)
                    data_objeto = datetime.strptime(data_encontrada, '%d/%m/%Y')
                    # data_formatada = data_objeto.strftime('%Y-%m-%d')
                    return data_objeto  # Retorne a data encontrada
    except Exception as e:
        print(f'Erro ao extrair data de vencimento da certidão: {e}')
        return False

In [335]:
def renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config):
    
    # Capturar a linha onde 'NOME_ARQUIVO' é igual a 'certidao_id'
    linha_no_cert_config = df_cert_config.loc[df_cert_config['CERTIDAO_ID'] == certidao_id]

    if not linha_no_cert_config.empty:
            # nome referencia
            nome_ref = linha_no_cert_config['NOME_ARQUIVO'].values[0]
            
            arquivos_na_pasta = os.listdir(pasta_destino)

            for arquivo in arquivos_na_pasta:
                if nome_arquivo_monitor in arquivo:

                    indice_nome = nome_arquivo_monitor.find(nome_ref)
                    if indice_nome != -1:
                        print(f'nome original: {nome_arquivo_monitor}')
                        nome_recortado = nome_arquivo_monitor[indice_nome:]
                        print("Resultado:", nome_recortado)
                        
                        # novo nome = data de vencimento e nome recortado
                        data_vencimento_str = data_vencimento.strftime('%Y-%m-%d')
                        novo_nome = f"{data_vencimento_str}_{nome_recortado}"
                        novo_path = os.path.join(pasta_destino, novo_nome)
                        
                        # Caminho do arquivo antigo
                        arquivo_path = os.path.join(pasta_destino, nome_arquivo_monitor)
                        
                        # Verifica se o novo nome já existe na pasta, incrementa o contador se necessário
                        contador = 1
                        while os.path.exists(novo_path):
                            # Se o arquivo já existe, divide o nome e extensão e adiciona o contador
                            novo_nome = f'{data_vencimento_str}({contador})_{nome_recortado}'
                            novo_path = os.path.join(pasta_destino, novo_nome)
                            print(novo_path)
                            contador += 1

                        # Renomeia o arquivo antigo para o novo nome
                        try:
                            os.rename(arquivo_path, novo_path)
                            print(f"Arquivo renomeado para: {novo_path}")
                            return novo_nome
                        except FileNotFoundError:
                            print(f"Arquivo original '{arquivo_path}' não encontrado.")
                        except Exception as e:
                            print(f"Erro ao renomear o arquivo: {e}")

In [ ]:
def processa_monitor(df_monitor):
    horario_atual = datetime.now()

    # Itera no DF 
    for index, row in df_monitor.iterrows():
        if row['STATUS'] == 'adicionado':
            pdf_path = os.path.join(row['PASTA_DESTINO'], row['ARQUIVO'])
            print(pdf_path)

            # Inicializa variáveis para cada coluna que pode ser preenchida
            pasta_destino = row['PASTA_DESTINO']
            certidao_id = row['CERTIDAO_ID']
            nome_arquivo_monitor = row['ARQUIVO']
            created_at = row['CREATED_AT']
            data_inicio = row['DATA_INICIO']
            numero_certidao = row['NUMERO_CERTIDAO']
            nada_consta = row['NADA_CONSTA']
            data_vencimento = row['DATA_VENCIMENTO']

            # Flag de erro para controlar a execução geral
            error_flag = 0  # Começa com 0, indicando que tudo está bem

            try:
                # Abre o PDF e extrai o texto
                with pdfplumber.open(pdf_path) as pdf:
                    texto_pdf = ''.join(page.extract_text() or '' for page in pdf.pages)

                # Verifica e preenche cada coluna individualmente se estiver vazia

                # DATA DE CRIAÇÃO
                if pd.isna(row['CREATED_AT']) or row['CREATED_AT'] == 'nan':
                    try:
                        created_at = data_criacao_pdf(pdf_path)
                        print(f'created_at: {created_at}')
                        df_monitor.at[index, 'CREATED_AT'] = created_at

                    except Exception as e:
                        print(f"Erro ao extrair created_at: {e}")

                # DATA DE INÍCIO
                if pd.isna(row['DATA_INICIO']) or row['DATA_INICIO'] == 'nan':
                    try:
                        data_inicio = extrair_data_inicio(texto_pdf)
                        print(f'data_inicio: {data_inicio}')
                        df_monitor.at[index, 'DATA_INICIO'] = data_inicio

                    except Exception as e:
                        print(f"Erro ao extrair data_inicio: {e}")
                
                # NÚMERO DA CERTIDÃO
                if pd.isna(row['NUMERO_CERTIDAO']) or row['NUMERO_CERTIDAO'] == 'nan':
                    try:
                        numero_certidao = extrair_numero_certidao(texto_pdf, row)
                        print(f'numero_certidao: {numero_certidao}')
                    except Exception as e:
                        print(f"Erro ao extrair numero_certidao: {e}")

                # NADA CONSTA
                if pd.isna(row['NADA_CONSTA']) or row['NADA_CONSTA'] == 'nan':
                    try:
                        nada_consta = extrair_nada_consta(texto_pdf)
                        print(f'nada_consta: {nada_consta}')
                    except Exception as e:
                        print(f"Erro ao extrair nada_consta: {e}")

                # DATA DE VENCIMENTO
                if pd.isna(row['DATA_VENCIMENTO']) or row['DATA_VENCIMENTO'] == 'nan':
                    try:
                        data_vencimento = extrair_data_vencimento(texto_pdf, data_inicio, row)
                        print(f'data_vencimento: {data_vencimento}')
                    except Exception as e:
                        print(f"Erro ao extrair data_vencimento: {e}")

                # RENOMEANDO ARQUIVO
                novo_nome = renomear_pdf_path(data_vencimento, certidao_id, pasta_destino, nome_arquivo_monitor, df_cert_config)



            except Exception as e:
                print(f"Erro ao processar o arquivo {pdf_path}: {e}")

            finally:
                df_monitor.at[index, 'STATUS'] = 'monitorando'
                df_monitor.at[index, 'DATA_VENCIMENTO'] = data_vencimento
                df_monitor.at[index, 'NUMERO_CERTIDAO'] = numero_certidao
                df_monitor.at[index, 'NADA_CONSTA'] = nada_consta
                df_monitor.at[index, 'UPDATED_AT'] = horario_atual
                if novo_nome == None:
                    continue
                else:
                    df_monitor.at[index, 'ARQUIVO'] = novo_nome
                
                # if error_flag == 1:
                #     print('error flag 1')
                # else:
                #     print(f"Atualização parcial realizada para o index {index}, devido ao erro_flag.")

    return df_monitor
    

df_monitor = processa_monitor(df_monitor)

df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      55 non-null     object        
 1   CLIENTE_ID       55 non-null     object        
 2   PASTA_DESTINO    55 non-null     object        
 3   ARQUIVO          55 non-null     object        
 4   STATUS           55 non-null     object        
 5   DATA_INICIO      55 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  55 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  55 non-null     object        
 8   NADA_CONSTA      55 non-null     bool          
 9   CREATED_AT       55 non-null     datetime64[ns]
 10  UPDATED_AT       55 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 4.5+ KB


In [337]:
df_monitor.head(100)

,CERTIDAO_ID,CLIENTE_ID,PASTA_DESTINO,ARQUIVO,STATUS,DATA_INICIO,DATA_VENCIMENTO,NUMERO_CERTIDAO,NADA_CONSTA,CREATED_AT,UPDATED_AT
0,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-19_CR FGTS.pdf,monitorando,2024-10-21,2024-11-19,2024102105285887458243,True,2024-11-13 20:08:57,2024-11-13 22:07:17.242
1,CEF-CRFGTS,AUTOLTDA,clientes/autodata/,2024-11-08_CR FGTS.pdf,VENCIDA,2024-10-03,2024-11-08,J0ER9QQR4U,True,2024-11-13 20:08:58,2024-11-14 01:34:47.709
2,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-12-01_CN Falencia e Recuperacao.pdf,monitorando,2024-11-01,2024-12-01,2024.CTD.ZZQ3.G1NW.0B85.GEDK.UR8P,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601
3,TJDFT-CNFR,AUTOLTDA,clientes/autodata/,2024-11-04_CN Falencia e Recuperacao.pdf.pdf,VENCIDA,2024-10-05,2024-11-04,091O5ZAMKM,True,2024-11-13 20:08:58,2024-11-14 01:34:47.709
4,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-12-04_CN Correcional (ePAD, CGU-PJ, CEIS,...",monitorando,2024-11-04,2024-12-04,YLkevKTcTH7aFG1S8seQ,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601
5,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-11-07_CN Correcional (ePAD, CGU-PJ, CEIS,...",VENCIDA,2024-10-02,2024-11-07,16P7NA13TV,True,2024-11-13 20:08:58,2024-11-14 01:34:47.709
6,CGU-CNC,AUTOLTDA,clientes/autodata/,"2024-11-07_CN Correcional (ePAD, CGU-PJ, CEIS,...",VENCIDA,2024-10-02,2024-11-07,16P7NA13TV,True,2024-11-13 20:08:58,2024-11-14 01:34:47.710
7,FN-CNDTF,AUTOLTDA,clientes/autodata/,2025-04-14_CND Tributos Federais - Copia.pdf,monitorando,2024-10-16,2025-04-14,2D3C.4DB5.AC29.EEAB,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601
8,FN-CNDTF,AUTOLTDA,clientes/autodata/,2025-04-14_CND Tributos Federais.pdf,monitorando,2024-10-16,2025-04-14,2D3C.4DB5.AC29.EEAB,True,2024-11-13 20:08:57,2024-11-13 22:08:31.601
9,FN-CNDTF,AUTOLTDA,clientes/autodata/,2024-11-08_CND Tributos Federais.pdf,VENCIDA,2024-10-04,2024-11-08,RY84ZPBE1B,True,2024-11-13 20:08:58,2024-11-14 01:34:47.710


In [ ]:
#### BUG, SE pode renomar o próximo errado caso ñ encontre o arquivo a ser procurado pois a variavel continua com o mesmo valor

In [338]:
df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      55 non-null     object        
 1   CLIENTE_ID       55 non-null     object        
 2   PASTA_DESTINO    55 non-null     object        
 3   ARQUIVO          55 non-null     object        
 4   STATUS           55 non-null     object        
 5   DATA_INICIO      55 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  55 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  55 non-null     object        
 8   NADA_CONSTA      55 non-null     bool          
 9   CREATED_AT       55 non-null     datetime64[ns]
 10  UPDATED_AT       55 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 4.5+ KB


In [339]:
df_monitor['NADA_CONSTA'] = df_monitor['NADA_CONSTA'].astype(bool)

In [341]:
from openpyxl import load_workbook

def atualizar_planilha(gestao_monitoramento, workbook, df_monitor):
    # Carrega o arquivo Excel
    workbook = load_workbook(gestao_monitoramento)
    
    # Seleciona a aba 'db_monitor'
    sheet = workbook['db_monitor']

    # Localiza a tabela 'fDB_monitor'
    table = None
    for tbl in sheet._tables.values():
        if tbl.name == 'fDB_monitor':
            table = tbl
            break

    if table is None:
        print("Tabela 'fDB_monitor' não encontrada.")
        return

    # Limpa todas as linhas da tabela (exceto o cabeçalho)
    start_row = table.ref.split(':')[0][1:]
    end_row = table.ref.split(':')[1][1:]
    start_col = table.ref.split(':')[0][0]
    end_col = table.ref.split(':')[1][0]

    for row in sheet.iter_rows(min_row=int(start_row) + 1, max_row=int(end_row), min_col=ord(start_col) - 64, max_col=ord(end_col) - 64):
        for cell in row:
            cell.value = None

    # Preenche os dados do DataFrame a partir da linha da tabela
    for index, row in df_monitor.iterrows():
        for col_num, value in enumerate(row, start=1):  # start=1 para a primeira coluna (A)
            sheet.cell(row=int(start_row) + 1 + index, column=col_num, value=value)  # Inicia na linha da tabela, logo abaixo do cabeçalho

    # Atualiza o intervalo da tabela para incluir todas as novas linhas
    table.ref = f"{start_col}{start_row}:{end_col}{int(start_row) + len(df_monitor)}"

    # Salva as alterações no arquivo Excel
    workbook.save(gestao_monitoramento)
    print("Arquivo Excel atualizado com os dados do DataFrame!")

# Exemplo de uso
gestao_monitoramento = 'gestao_monitoramento.xlsx'
workbook = load_workbook(gestao_monitoramento)
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')

atualizar_planilha(gestao_monitoramento, workbook, df_monitor)


Arquivo Excel atualizado com os dados do DataFrame!


In [331]:
atualizar_planilha(gestao_monitoramento, workbook, df_monitor)


Arquivo Excel atualizado com os dados do DataFrame!


# CHECAGEM STATUS - Vencidas

In [326]:
df_monitor = pd.read_excel(gestao_monitoramento, sheet_name='db_monitor')
df_monitor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CERTIDAO_ID      55 non-null     object        
 1   CLIENTE_ID       55 non-null     object        
 2   PASTA_DESTINO    55 non-null     object        
 3   ARQUIVO          55 non-null     object        
 4   STATUS           55 non-null     object        
 5   DATA_INICIO      55 non-null     datetime64[ns]
 6   DATA_VENCIMENTO  55 non-null     datetime64[ns]
 7   NUMERO_CERTIDAO  55 non-null     object        
 8   NADA_CONSTA      55 non-null     bool          
 9   CREATED_AT       55 non-null     datetime64[ns]
 10  UPDATED_AT       55 non-null     datetime64[ns]
dtypes: bool(1), datetime64[ns](4), object(6)
memory usage: 4.5+ KB


In [328]:
for index, row in df_monitor.iterrows():
    horario_atual = datetime.now()
    if row['DATA_VENCIMENTO'] <= horario_atual:
        df_monitor.at[index, 'STATUS'] = 'VENCIDA'
        df_monitor.at[index, 'UPDATED_AT'] = horario_atual
        


# RELATÓRIO

O que deve ter no relatório:
* nº de novas certidões encontradas
* quais certidões estão faltando
** aqui, trazer lista de quais são, 
* nº de 